#  T-tests and P-values

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_rel`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](http://b.link/test50) and [this Quora](http://b.link/unpaired97). Make sure you understand when to use which type of t-test. 

In [73]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import scipy.stats as st

### One tailed t-test 
In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/ttest_machine.xlsx. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

#### Import dataset

In [84]:
data = pd.read_csv('ttest_machine.txt')
machine = pd.DataFrame(data)

display(machine)

,New_machine,Old_machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [ ]:
# H0 = Old machine is quicker at packing 10 cartons
# H1 = New machine is quicker at packing 10 cartons

In [85]:
new_machine = machine['New_machine'].sample(10)
old_machine = machine['Old_machine'].sample(10)

display(new_machine)
display(old_machine)

0    42.1
8    41.8
9    42.7
2    41.3
4    42.4
6    43.2
1    41.0
5    42.8
7    42.3
3    41.8
Name: New_machine, dtype: float64

7    41.7
6    43.1
4    42.5
0    42.7
5    43.5
2    43.8
1    43.6
8    44.0
9    44.1
3    43.3
Name: Old_machine, dtype: float64

In [86]:
st.ttest_ind(new_machine, old_machine)

TtestResult(statistic=-3.3972307061176026, pvalue=0.0032111425007745158, df=18.0)

In [87]:
t_statistic, p_value = ttest_ind(new_machine, old_machine, alternative='less')

In [88]:
print(f'T-statistic: {t_statistic}')
print(f'One-tailed p-value: {p_value / 2}') 

alpha = 0.05
if p_value / 2 < alpha:
    print('Reject the null hypothesis. New machine is faster.')
else:
    print('Not enough evidence to reject the null hypothesis.')

T-statistic: -3.3972307061176026
One-tailed p-value: 0.0008027856251936289
Reject the null hypothesis. New machine is faster.


In [83]:
# The p-value is below the 0.05 threshold based on the sample size.
# There is evidence to suggest that the new machine will pack 10 cartons quicker.

#### Import dataset

In this challenge we will work on the Pokemon dataset you have already used. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.). Use pokemon.csv

In [89]:
data = pd.read_csv('pokemon.txt')
pokemon = pd.DataFrame(data)

display(pokemon)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [96]:
def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    results = {}

    for feature in features:
        t_statistic, p_value = ttest_ind(s1[feature], s2[feature])
        results[feature] = {'t_statistic': t_statistic, 'p_value': p_value}
        
    return results

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [105]:
sample1 = pokemon[pokemon['Legendary'] == True]
sample2 = pokemon[pokemon['Legendary'] == False]

results = t_test_features(sample1, sample2)
display(results)

{'HP': {'t_statistic': 8.036124405043928, 'p_value': 3.330647684846191e-15},
 'Attack': {'t_statistic': 10.397321023700622,
  'p_value': 7.827253003205333e-24},
 'Defense': {'t_statistic': 7.181240122992339,
  'p_value': 1.5842226094427255e-12},
 'Sp. Atk': {'t_statistic': 14.191406210846289,
  'p_value': 6.314915770427266e-41},
 'Sp. Def': {'t_statistic': 11.03775106120522,
  'p_value': 1.8439809580409594e-26},
 'Speed': {'t_statistic': 9.765234331931898,
  'p_value': 2.3540754436898437e-21},
 'Total': {'t_statistic': 16.386116965872425,
  'p_value': 3.0952457469652825e-52}}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [ ]:
# We can see the p-value being signficantly lower than the 0.05 threshold across all features.
# The null hypothesis is rejected and can state that the stats vary significantly different.

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [106]:
generation1 = pokemon[pokemon['Generation'] == 1]
generation2 = pokemon[pokemon['Generation'] == 2]

results = t_test_features(generation1, generation2)
display(results)

{'HP': {'t_statistic': -1.4879963792658193, 'p_value': 0.13791881412813622},
 'Attack': {'t_statistic': 1.1763035045816723, 'p_value': 0.24050968418101445},
 'Defense': {'t_statistic': -0.6124386271679768,
  'p_value': 0.5407630349194362},
 'Sp. Atk': {'t_statistic': 1.475669371406143, 'p_value': 0.14119788176331508},
 'Sp. Def': {'t_statistic': -1.3829844029533516,
  'p_value': 0.1678122623160639},
 'Speed': {'t_statistic': 3.0126333688329856,
  'p_value': 0.0028356954812578704},
 'Total': {'t_statistic': 0.5836931395106778, 'p_value': 0.5599140649014442}}

#### What conclusions can you make?

In [ ]:
# The p-value varies across some of the features
# All features are higher than the 0.5 threshold, other than speed.
# The assumption is that for majority of the features are not that different from Gen 1 and 2.

#### Compare pokemons who have single type vs those having two types.

In [107]:
type1 = pokemon[(~pokemon['Type 1'].isna()) & (~pokemon['Type 2'].isna())]
type2 = pokemon[(~pokemon['Type 1'].isna()) & pokemon['Type 2'].isna()]

results = t_test_features(type1, type2)
display(results)

{'HP': {'t_statistic': 1.597247491548339, 'p_value': 0.11060643144431842},
 'Attack': {'t_statistic': 3.7972414565658683,
  'p_value': 0.00015741395666164396},
 'Defense': {'t_statistic': 5.58247971732322,
  'p_value': 3.250594205757004e-08},
 'Sp. Atk': {'t_statistic': 3.8170824160380157,
  'p_value': 0.0001454917404035147},
 'Sp. Def': {'t_statistic': 3.8892352214769463,
  'p_value': 0.00010893304795534396},
 'Speed': {'t_statistic': 2.2606203667533133, 'p_value': 0.02405141079403746},
 'Total': {'t_statistic': 5.345970873144627,
  'p_value': 1.1749035008828752e-07}}

#### What conclusions can you make?

In [ ]:
# There are certain features being a two type Pokemone will have significantly different stats.
# These features are HP, Defense and Total.
# No big difference between Special abilities (Attack and Defense) and Speed... just.

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [112]:
attdef1 = pokemon.dropna(subset=['Attack', 'Defense'])
attdef1['Difference'] = attdef1['Attack'] - attdef1['Defense']

attdef2 = pokemon.dropna(subset=['Sp. Atk', 'Sp. Def'])
attdef2['Difference'] = attdef2['Sp. Atk'] - attdef2['Sp. Def']

results = t_test_features(attdef1, attdef2, features=['Difference'])
display(results)

{'Difference': {'t_statistic': 2.642219930301883,
  'p_value': 0.008316859771593346}}

#### What conclusions can you make?

In [ ]:
# There is no signficant difference between the normal Attack/Defense and the Special Attack/Defense.